In [1]:
import tensorflow as tf

# string_input_producer
creates a FIFO queue for holding the filenames until the reader needs them.
## shuffle=False
has options for shuffling and setting a maximum number of epochs. 
A queue runner adds the whole list of filenames to the queue once for each epoch, shuffling the filenames within an epoch if shuffle=True.

Each execution of read reads a single line from the file. 


In [2]:
import numpy as np
import time
 
start = time.clock()

filename_queue = tf.train.string_input_producer(
   ['data.csv'], shuffle=False, name='filename_queue')

# read data by line with the file queue
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. 
Also specifies the type of the decoded result.

The decode_csv op then parses the result into a list of tensors.

The record_defaults argument determines the type of the resulting tensors and sets the default value to use if a value is missing in the input string.


In [3]:
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
# x_data = xy[:, 0:-1]
# y_data = xy[:,[-1]]  ##### error
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [4]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables (like the epoch counter) in the graph.
sess.run(tf.global_variables_initializer())


# tf.train.Coordinator

This is responsible for letting all the threads know if anything has signalled a shut down. 


#  tf.train.start_queue_runners 

to populate the queue before you call run or eval to execute the read.

Otherwise read will block(挂起) while it waits for filenames from the queue.

https://www.tensorflow.org/programmers_guide/threading_and_queues

In [5]:
# Start input enqueue threads.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [6]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], 
        feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, 
                   "\nPrediction:\n", hy_val)

        
elapsed = (time.clock() - start)
print("Time used:",elapsed)


0 Cost:  87025.8 
Prediction:
 [[-124.49518585]
 [-162.18504333]
 [-153.40196228]
 [-167.80088806]
 [-125.92416382]
 [ -94.87765503]
 [-122.73983765]
 [ -86.74608612]
 [-155.08970642]
 [-146.01365662]]
10 Cost:  59.3584 
Prediction:
 [[ 158.21568298]
 [ 177.71166992]
 [ 181.44830322]
 [ 196.89750671]
 [ 133.31103516]
 [  96.29056549]
 [ 150.25170898]
 [ 115.63300323]
 [ 162.6178894 ]
 [ 148.31671143]]
20 Cost:  47.8159 
Prediction:
 [[ 160.09718323]
 [ 180.02285767]
 [ 183.69908142]
 [ 199.35865784]
 [ 135.07789612]
 [  97.6142807 ]
 [ 152.09469604]
 [ 116.9988327 ]
 [ 164.80148315]
 [ 150.36557007]]
30 Cost:  47.4026 
Prediction:
 [[ 160.08369446]
 [ 180.05587769]
 [ 183.70544434]
 [ 199.37535095]
 [ 135.10723877]
 [  97.65672302]
 [ 152.1078949 ]
 [ 117.00814819]
 [ 164.85568237]
 [ 150.4417572 ]]
40 Cost:  47.041 
Prediction:
 [[ 160.05752563]
 [ 180.07345581]
 [ 183.69671631]
 [ 199.37556458]
 [ 135.12481689]
 [  97.69038391]
 [ 152.10873413]
 [ 117.00828552]
 [ 164.89535522]
 [ 15

# Wait for threads to finish.

In [7]:
coord.request_stop()
coord.join(threads)

In [8]:
import tensorflow as tf
import numpy as np
import time
 
start = time.clock()

filename_queue = tf.train.string_input_producer(
   ['data.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
   x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
   cost_val, hy_val, _ = sess.run(
       [cost, hypothesis, train], 
       feed_dict={X: x_batch, Y: y_batch})
   if step % 10 == 0:
       print(step, "Cost: ", cost_val, 
                   "\nPrediction:\n", hy_val)

        
elapsed = (time.clock() - start)
print("Time used:",elapsed)


coord.request_stop()
coord.join(threads)


0 Cost:  67105.6 
Prediction:
 [[-105.53936768]
 [-115.94915771]
 [-119.98350525]
 [-129.18899536]
 [ -86.94707489]
 [ -60.54927444]
 [ -97.56317902]
 [ -73.85138702]
 [-104.02748871]
 [ -92.10940552]]
10 Cost:  26.0802 
Prediction:
 [[ 146.9294281 ]
 [ 187.50993347]
 [ 179.01142883]
 [ 196.44621277]
 [ 144.48780823]
 [ 110.09085846]
 [ 146.19250488]
 [ 106.86264801]
 [ 179.58869934]
 [ 170.60508728]]
20 Cost:  32.1334 
Prediction:
 [[ 148.64967346]
 [ 189.54399109]
 [ 181.03311157]
 [ 198.6451416 ]
 [ 146.03312683]
 [ 111.22185516]
 [ 147.84362793]
 [ 108.09493256]
 [ 181.47840881]
 [ 172.34613037]]
30 Cost:  32.0151 
Prediction:
 [[ 148.67774963]
 [ 189.54400635]
 [ 181.05075073]
 [ 198.66145325]
 [ 146.02716064]
 [ 111.20903015]
 [ 147.86099243]
 [ 108.11599731]
 [ 181.46707153]
 [ 172.32620239]]
40 Cost:  31.8356 
Prediction:
 [[ 148.69433594]
 [ 189.53034973]
 [ 181.05484009]
 [ 198.6630249 ]
 [ 146.01078796]
 [ 111.18857574]
 [ 147.86732483]
 [ 108.12887573]
 [ 181.44300842]
 [ 1